In [5]:
import os
#-------------------------------------------------------------------------------
def rename_images(paths):
    for path in paths:
        for i,filename in enumerate(os.listdir(path)):
            os.rename(path+filename, filename.split('_')[1]+'_'+str(i)+'.jpg')
#-------------------------------------------------------------------------------
def combine_images(img_path1,img_path2):
    image_paths = os.listdir(img_path1) + os.listdir(img_path2)
    labels = [0] * len(os.listdir(img_path1)) + [1] * len(os.listdir(img_path2))
    return image_paths,labels
#-------------------------------------------------------------------------------

In [7]:
# rename images
rename_images(['./images/hotdog/','./images/notdog/'])

In [8]:
combined_path,labels = combine_images('./images/hotdog/','./images/notdog/')

In [15]:
import cv2 as cv
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# get descriptors (feature extraction)
def sift(img_path):
    img = cv.imread(img_path)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp,desc = sift.detectAndCompute(gray,None) #tuple,nd-array
    return kp,desc
#-------------------------------------------------------------------------------
# KNN clustering into VWs
def cluster(features,K):
    kmeans = KMeans(n_clusters=K)
    features = features.reshape(-1, 1)
    kmeans.fit(features)
    return kmeans.cluster_centers_
#-------------------------------------------------------------------------------
# histogram
def histogram(img_path,words,kmeans):
    kp,desc = sift(img_path=img_path)

    word = kmeans.predict(desc)

    hist = np.zeros(len(words))
    for word in words:
        hist[word]+=1
    hist = hist / len(kp)
    return hist
# train
#-------------------------------------------------------------------------------
# evaluate
def train(img_paths,labels,K):
    features = []
    for img in img_paths:
        _,desc = sift(img)
        features.append(desc)
    words = cluster(features=features,K=K)
    kmeans = KMeans(n_clusters=K)
    kmeans.cluster_centers_ = words

    histogram_list = []
    label_list = []

    for im,label in zip(img_paths,labels):
        histogram_list.append(histogram(im,words,kmeans))
        label_list.append(label)

    X_train,X_test,Y_train,Y_test = train_test_split(histogram_list,label_list,test_size=0.2)

    svm = SVC(kernel='linear')
    svm.fit(X_train,Y_train)

    accuracy = svm.score(X_test,Y_test)
    print('Accuracy: {:.2f}%'.format(accuracy*100))
    return svm
#-------------------------------------------------------------------------------
# predict
# def predict(path,words,kmeans,classifier):
#     histogram_list = []
#     for im in path:
#         histogram_list.append(histogram(im,words,kmeans))

#     preds = classifier.predict(histogram_list)

#     for im,pred in zip(path,preds):

In [16]:
train(combined_path,labels,2)

AttributeError: 'list' object has no attribute 'reshape'